In [3]:
import numpy as np

In [5]:
X1 = np.load("Test_data\\CIFAR2\\hog_test.npy")
X2 = np.load("Test_data\\CIFAR2\\3x3_test.npy")
X3 = np.load("Test_data\\CIFAR2\\4x4_test.npy")
y = np.load("Test_data\\CIFAR2\\y_test.npy")

In [6]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
def quantile_binning(data, n_bins=4):
    
    binned_data = []
    for column in data.columns:
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
        column_binned = discretizer.fit_transform(data[[column]]).astype(int)
        binned_data.append(column_binned)

    binned_data = np.hstack(binned_data)
    return pd.DataFrame(binned_data, columns=data.columns)

def booleanize(binned_data):
    
    booleanized_data = []
    column_names = []

    for column in binned_data.columns:
        unique_bins = sorted(binned_data[column].unique())  
        for bin_value in unique_bins:
            boolean_column = (binned_data[column] == bin_value).astype(int)
            booleanized_data.append(boolean_column)
            column_names.append(f"{column}_bin_{bin_value}")
    

    booleanized_data = np.column_stack(booleanized_data)
    return pd.DataFrame(booleanized_data, columns=column_names)


def booleanize_automated(num_array,binn):
    num_array=pd.DataFrame(num_array)
    num_bin=quantile_binning(num_array,binn)
    num_final=booleanize(num_bin)
    return num_final


In [8]:
features1 = {
    "X1": X1,
    "X2": X2,
    "X3": X3,
}

def booleanize_all(features_dict, skip_keys=None, binn=5):
    skip_keys = skip_keys or []
    for key in features_dict:
        if key in skip_keys:
            if hasattr(features_dict[key], "to_numpy"):
                features_dict[key] = features_dict[key].to_numpy()
        else:
            features_dict[key] = booleanize_automated(features_dict[key], binn=binn)
            if hasattr(features_dict[key], "to_numpy"):
                features_dict[key] = features_dict[key].to_numpy()
    return features_dict

features = booleanize_all(features1, binn=5)

d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
d:\Anaconda Projects\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be ch

In [9]:
features["X1"]

array([[0, 0, 1, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], shape=(10000, 1620))

In [6]:
y

array([7, 2, 1, ..., 4, 5, 6], shape=(10000,), dtype=uint8)

In [11]:
def save_data_to_txt(X_bool: np.ndarray, y: np.ndarray, filename: str):
    with open(filename, 'w') as f:
        for i in range(len(y)):
            line = ' '.join(map(str, X_bool[i])) + f' {y[i]}\n'
            f.write(line)

# Example usage:
save_data_to_txt(features["X1"], y, 'Test_data\\CIFAR2\\hog_test.txt')
save_data_to_txt(features["X2"], y, 'Test_data\\CIFAR2\\3x3_test.txt')
save_data_to_txt(features["X3"], y, 'Test_data\\CIFAR2\\4x4_test.txt')
